In [20]:
import pandas as pd
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
import ollama
from langchain.llms import Ollama
from langchain.chains import LLMChain

In [4]:
fewShotData = pd.read_csv('../Training Dataset/FewShotForEvents.csv')
fewShotData = fewShotData[['content', 'events refined']]
fewShotData = fewShotData.dropna()
fewShotData.head()

,content,events refined
0,Keeps logging me out of account saying session...,"[It keeps logging me out, It is saying session..."
1,Facebook marketplace has a bug or a terrible f...,"[I search and scroll, It will select a random ..."
2,I don't know what has happened lately it will ...,"[I log in with the right password, The app sta..."
3,Can't login at all can't recover my account at...,"[I have tried uninstalling, I tried to get the..."
4,When I try to share things that I like on my w...,"[I try to share things, it throws it in the ir..."


In [19]:
few_shot_examples = fewShotData.to_dict(orient='records')

# Define the Example PromptTemplate
examplePrompt = PromptTemplate(
    input_variables=['content', 'events refined'],
    template="Reviews: {content}\nEvents: {events refined}\n"
)

# Create the FewShotPromptTemplate
fewShotPrompt = FewShotPromptTemplate(
    examples=few_shot_examples,
    example_prompt=examplePrompt,
    suffix="Reviews: {content}",
    input_variables=['content']
)

# Generate and format the prompt
formatted_prompt = fewShotPrompt.format(content="I went to the store today.")
print(formatted_prompt)

Reviews: Keeps logging me out of account saying session expired and everytime I try to log in it won't let me, keeps updating constantly even when I keep it up to date so not sure if the app is glitching, overall not a good app so may have to consider deleting Facebook as this has put me off and happens way too much, keep trying to update the app thinking it was that but its still not working and it says update again even after i already did
Events: [It keeps logging me out, It is saying session expired, I try to log in, It won't let me log in][I keep trying to update the app, It still won't let me log in][I update app, It says update again]


Reviews: Facebook marketplace has a bug or a terrible feature where it will select a random picture as the main one (so the only one buyers see when searching and scrolling) keeps selecting a zoomed up tiny area of damage as the main photo instead of the actual overall photo of an object no wonder no one but scam bots message me now.
Events: [I s

In [16]:
# ollama.list()

{'models': [{'name': 'fewShotLLAMA2:latest',
   'model': 'fewShotLLAMA2:latest',
   'modified_at': '2024-06-01T03:07:08.2162239+06:00',
   'size': 3826794200,
   'digest': '1e28f7db7135fece538a276a7fad66290310b6db7383a44bd0a9065a573191de',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '6.7B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'},
  {'name': 'EventExtraction-llama2:latest',
   'model': 'EventExtraction-llama2:latest',
   'modified_at': '2024-05-31T02:42:47.3735996+06:00',
   'size': 3826794719,
   'digest': '4c7872e5e274c752b29d74d425a228af534ee19be33ea4c149da09ad4add9d80',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '6.7B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'},
  {'name': 'llama2:latest',
   'model': 'llama2:latest',
   'modified_at': '2024-05-

In [24]:
class CustomOllamaLLM(Ollama):
    def __init__(self, base_url, model):
        self.base_url = base_url
        self.model = model
    
    def _call(self, prompt, stop=None):
        # Implement the call to your local server here
        # For demonstration, returning a placeholder response
        response = "Predicted Event: Placeholder"
        return response

    @property
    def _identifying_params(self):
        return {"base_url": self.base_url, "model": self.model}


In [25]:
ollama = Ollama(base_url='http://localhost:11434', model='fewShotLLAMA2')

In [26]:
llamaChain = LLMChain(llm=ollama, prompt=fewShotPrompt)

In [27]:
result = llamaChain.run({"content": "I went to the store today."})
print("New Content:", "I went to the store today.")
print("Predicted Event:", result)

d:\Academics\Thesis\New folder\Web-Scraping\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


New Content: I went to the store today.
Predicted Event: 
Here are the reviews from the text you provided:

1. "Need to include a list feature in update section of status viewers , the app including many part of a individuals life status viewer can separated using list connecting different life . Also adding scheduling feature for Channel is also very helpfull after rolling our new features the app has becoming bit lagging which usually disruption to workflows ."
2. "Not bad, but y'all gotta fix this issue whenever I upload a status there's a few who works with sound on as it should but a few others doesn't, all the videos has the same quality and voices the issue is in this app fix it as soon as possible."
3. "⛔Must Read⛔ I am using whatsapp since 2018 but i facing some trouble with latest update when i make call to someone it showing me Ringing even my or Receiver's mobile phone has disabled data or wifi not connected any wifi, in simple it show me Ringing even Reciver or me offline 

In [28]:
print(result)


Here are the reviews from the text you provided:

1. "Need to include a list feature in update section of status viewers , the app including many part of a individuals life status viewer can separated using list connecting different life . Also adding scheduling feature for Channel is also very helpfull after rolling our new features the app has becoming bit lagging which usually disruption to workflows ."
2. "Not bad, but y'all gotta fix this issue whenever I upload a status there's a few who works with sound on as it should but a few others doesn't, all the videos has the same quality and voices the issue is in this app fix it as soon as possible."
3. "⛔Must Read⛔ I am using whatsapp since 2018 but i facing some trouble with latest update when i make call to someone it showing me Ringing even my or Receiver's mobile phone has disabled data or wifi not connected any wifi, in simple it show me Ringing even Reciver or me offline but in previous it show me calling and when receiver phon